# Neural Networks in Python - 3 ways to get you started!

In this tutorial, we will implement a multi-layered perceptron (a type of a feed-forward neural network) in Python using three different libraries. We’ll start off with the most basic example possible, going to more complex and flexible frameworks with the aim of increasing our understanding of how to implement neural networks in Python.

For an indepth explanation see the blog post at Medium.com:

https://medium.com/@mattvonrohr/neural-networks-in-python-3-ways-to-get-started-with-multi-layered-perceptrons-20c44e22ae05


# scikit-learn

In [1]:
!pip install scikit-learn

In [2]:
from sklearn.datasets import load_boston

boston = load_boston()
X = boston.data
y = boston.target

NUM_EPOCHS = 200
NUM_HIDDEN = 500
LR = 0.01

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def report(y_pred, y_true):
  rmse = np.sqrt(mean_squared_error(y_pred, y_true))
  r2 = r2_score(y_pred, y_true)
  print(f'RMSE: {rmse}\n R^2: {r2}')

In [4]:
reg = MLPRegressor(hidden_layer_sizes=[NUM_HIDDEN], max_iter=NUM_EPOCHS, learning_rate_init=LR)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

report(y_pred, y_test)

RMSE: 3.1618361558540142
 R^2: 0.8825391998928166


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Keras

In [5]:
!pip install keras

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        layers.Dense(13, activation="relu"),
        layers.Dense(NUM_HIDDEN, activation="relu"),
        layers.Dense(1),
    ]
)

optimizer = keras.optimizers.Adam(LR)
model.compile(loss='mse', optimizer=optimizer)

model.fit(X_train, y_train, epochs=NUM_EPOCHS, verbose=0)
y_pred = model.predict(X_test)

report(y_pred, y_test)

RMSE: 3.4203393829662945
 R^2: 0.8827584160124081


# PyTorch

In [7]:
!pip install torch

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, NUM_HIDDEN)
        self.fc2 = nn.Linear(NUM_HIDDEN, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Training on', device)
model = Net().to(device)

X_train_pt = torch.FloatTensor(X_train).to(device)
X_test_pt = torch.FloatTensor(X_test).to(device)
y_train_pt = torch.FloatTensor(y_train).reshape(-1, 1).to(device)

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for t in range(NUM_EPOCHS):
    optimizer.zero_grad()
    y_pred_pt = model(X_train_pt)
    loss = loss_function(y_pred_pt, y_train_pt)
    loss.backward()   
    optimizer.step()

    if t % 200 == 99:
        y_pred_pt = model(X_test_pt)
        print(t, loss.cpu().item())
        model.zero_grad()

y_pred = model(X_test_pt).reshape(-1).detach().cpu().numpy()
report(y_pred, y_test)

Training on cpu
99 9.207415580749512
RMSE: 3.458637881835001
 R^2: 0.854834632015996


Just for some fun, let's ensemble our predictions together with a GBM (which does have good performance out of the box) and further reduce the RMSE of our predictions:

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.025)
reg.fit(X_train, y_train)
y_pred_gbr = reg.predict(X_test)
report(y_pred_gbr, y_test)

RMSE: 3.1961037510686348
 R^2: 0.8668624822582959


In [10]:
y_pred_ensemble = (y_pred_gbr + y_pred) / 2
report(y_pred_ensemble, y_test)

RMSE: 2.968930748717139
 R^2: 0.8859568546619003


In [11]:
import pandas as pd
pd.DataFrame({
    'ensemble': y_pred_ensemble,
    'y_test': y_test
})  

,ensemble,y_test
0,34.409664,36.1
1,21.353057,19.8
2,9.876751,8.8
3,29.295288,30.8
4,22.182562,21.9
...,...,...
122,27.268824,23.7
123,40.602346,43.1
124,25.158633,23.3
125,17.611248,17.8
